##Problem 1 - CIFAR-100 Classification
Please change the Edit>Notebook>Hardware Accelerator settings in Colab from None to GPU.

In [ ]:
# Check the GPU information
!nvidia-smi

Sat Apr 22 19:46:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    32W /  70W |   7699MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# CIFAR-100 classification
import torch
import torchvision
from torchvision import transforms

# Define the batch size
batch_size = 256

# Define the transform to normalize the data
transform = torchvision.transforms.Compose([
    transforms.Resize((224, 224)), # Resize to 224x224 (height x width)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])


# Load the training set
train_set = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)

# Load the test set
test_set = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

# Create data loaders to load the data in batches
train_loader =  torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the model
from torchvision.models import resnet18, ResNet18_Weights

# Use pretrained weights
net = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).cuda()

In [ ]:
# Define the loss and the optimizer
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn

epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=0, last_epoch=- 1, verbose=False)

In [ ]:
# Train the network
acc_list = []
for epoch in range(epochs):  # Loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    scheduler.step()
            
    # Test the network
    net.eval()
    correct = 0
    total = 0
    # Since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            # Calculate outputs by running images through the network
            outputs = net(images)
            
            # The class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        acc_list.append(100 * correct // total)

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
print('Finished Training')

[1,     1] loss: 0.005
[1,    11] loss: 0.041
[1,    21] loss: 0.026
[1,    31] loss: 0.020
[1,    41] loss: 0.017
[1,    51] loss: 0.016
[1,    61] loss: 0.014
[1,    71] loss: 0.013
[1,    81] loss: 0.012
[1,    91] loss: 0.011
[1,   101] loss: 0.010
[1,   111] loss: 0.010
[1,   121] loss: 0.009
[1,   131] loss: 0.009
[1,   141] loss: 0.008
[1,   151] loss: 0.008
[1,   161] loss: 0.008
[1,   171] loss: 0.008
[1,   181] loss: 0.007
[1,   191] loss: 0.007
Accuracy of the network on the 10000 test images: 59 %
[2,     1] loss: 0.001
[2,    11] loss: 0.006
[2,    21] loss: 0.006
[2,    31] loss: 0.005
[2,    41] loss: 0.005
[2,    51] loss: 0.005
[2,    61] loss: 0.005
[2,    71] loss: 0.005
[2,    81] loss: 0.005
[2,    91] loss: 0.005
[2,   101] loss: 0.005
[2,   111] loss: 0.005
[2,   121] loss: 0.005
[2,   131] loss: 0.005
[2,   141] loss: 0.005
[2,   151] loss: 0.005
[2,   161] loss: 0.005
[2,   171] loss: 0.005
[2,   181] loss: 0.005
[2,   191] loss: 0.005
Accuracy of the network o

In [ ]:
# CIFAR-100 classification
import torch
import torchvision
from torchvision import transforms

# Define the batch size
batch_size = 256

# Define the transform to normalize the data
transform = torchvision.transforms.Compose([
    transforms.Resize((224, 224)), # Resize to 224x224 (height x width)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])


# Load the training set
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Load the test set
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Create data loaders to load the data in batches
train_loader =  torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 104726089.57it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
dataiter = iter(train_loader)
dataiter

In [ ]:
images, labels = next(dataiter)

In [ ]:
labels

tensor([7, 4, 3, 8, 3, 5, 5, 5, 5, 6, 9, 4, 3, 8, 4, 9, 6, 7, 6, 0, 3, 9, 0, 0,
        0, 8, 0, 5, 3, 9, 8, 2, 3, 7, 4, 5, 5, 9, 5, 6, 8, 9, 7, 9, 2, 9, 1, 6,
        6, 2, 6, 1, 4, 5, 5, 2, 6, 3, 7, 2, 1, 3, 3, 2, 6, 9, 5, 5, 4, 1, 6, 9,
        0, 1, 8, 3, 4, 2, 1, 8, 4, 1, 6, 7, 6, 6, 0, 5, 4, 5, 2, 6, 1, 1, 8, 0,
        0, 3, 7, 7, 2, 5, 2, 3, 8, 0, 4, 7, 1, 3, 1, 4, 4, 7, 1, 1, 9, 7, 9, 4,
        6, 5, 7, 2, 9, 0, 8, 4, 3, 5, 4, 0, 8, 4, 5, 2, 2, 8, 3, 2, 4, 5, 9, 2,
        1, 6, 9, 4, 6, 2, 3, 8, 5, 4, 8, 7, 9, 6, 0, 2, 4, 3, 5, 4, 9, 8, 2, 9,
        6, 3, 5, 2, 3, 6, 0, 4, 2, 9, 9, 6, 8, 1, 3, 6, 8, 2, 0, 6, 4, 1, 4, 4,
        6, 9, 4, 2, 4, 2, 6, 9, 9, 9, 2, 6, 4, 2, 2, 3, 9, 0, 5, 2, 7, 9, 0, 0,
        4, 7, 4, 4, 3, 0, 2, 8, 2, 4, 0, 8, 1, 7, 9, 8, 2, 9, 6, 1, 1, 5, 8, 3,
        2, 4, 9, 4, 0, 0, 2, 0, 0, 7, 2, 9, 4, 7, 2, 6])

In [ ]:
# Define the model
from torchvision.models import resnet18, ResNet18_Weights

# Use pretrained weights
net = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).cuda()

In [ ]:
# Define the loss and the optimizer
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn

epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=0, last_epoch=- 1, verbose=False)